# Analyzing When and Where Influenza outbreak Occur in U.S


by Berthin Bitja

Note :
[How to install Homebrew on OS X?](https://stackoverflow.com/questions/20381128/how-to-install-homebrew-on-os-x)
[Old version X-code](https://stackoverflow.com/questions/10335747/how-to-download-xcode-dmg-or-xip-file)

In [1]:
options(warn = -1) #ignore warnings

# IMPORTANT: This assumes that all packages in "Rstart.R" are installed,
# and the fonts "Source Sans Pro" and "Open Sans Condensed Bold" are installed
# via extrafont. If ggplot2 charts fail to render, you may need to change/remove the theme call.


source("Rstart.R")
library(ggmap)



sessionInfo()


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Registering fonts with R

Attaching package: ‘scales’

The following object is masked from ‘package:readr’:

    col_factor

Google Maps API Terms of Service: http://developers.google.com/maps/terms.
Please cite ggmap if you use it: see citation("ggmap") for details.


R version 3.4.1 (2017-06-30)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Sierra 10.12.6

Matrix products: default
BLAS: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libBLAS.dylib
LAPACK: /System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/Versions/A/libLAPACK.dylib

locale:
[1] en_CA.UTF-8/en_CA.UTF-8/en_CA.UTF-8/C/en_CA.UTF-8/en_CA.UTF-8

attached base packages:
[1] grid      stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] ggmap_2.7          stringr_1.2.0      digest_0.6.12      RColorBrewer_1.1-2
[5] scales_0.4.1       extrafont_0.17     ggplot2_2.2.1      dplyr_0.7.4.9000  
[9] readr_1.1.1       

loaded via a namespace (and not attached):
 [1] Rcpp_0.12.13.1    compiler_3.4.1    plyr_1.8.4        bindr_0.1        
 [5] bitops_1.0-6      tools_3.4.1       uuid_0.1-2        lattice_0.20-35  
 [9] jsonlite_1.5      evalua

# Processing the Data

In [51]:
library(cdcfluview)
library(statebins)
# current verison
# packageVersion("cdcfluview")

#df.flu <- get_flu_data(region = "census", sub_region = 1:9, data_source = "all", years = 2008:2015)
#df.flu.census <- get_flu_data(region = "census", sub_region = 1:10, data_source = "ilinet", years = 2008:2015)

# I am looking at the ILI activity
df.state <- get_state_data(years=2010:2015)

In [50]:
#display the data 
#df.flu %>% head(10)
#sprintf("# of Rows in Dataframe: %s", nrow(df.flu))
#sprintf("Dataframe Size: %s", format(object.size(df.flu), units = "MB"))

In [49]:
#display the data 
df.state %>% head(10)
sprintf("# of Rows in Dataframe: %s", nrow(df.state))
sprintf("Dataframe Size: %s", format(object.size(df.state), units = "MB"))

statename,url,website,activity_level,activity_level_label,weekend,season,weeknumber
District of Columbia,http://doh.dc.gov/page/influenza-season,Influenza Information,0,Insufficient Data,May-24-2014,2013-14,21
Connecticut,http://www.ct.gov/dph/cwp/view.asp?a=3136&q=410788,Flu Statistics,0,Insufficient Data,May-24-2014,2013-14,21
District of Columbia,http://doh.dc.gov/page/influenza-season,Influenza Information,0,Insufficient Data,May-31-2014,2013-14,22
Puerto Rico,http://www.salud.gov.pr/Estadisticas-Registros-y-Publicaciones/Pages/Influenza.aspx,Puerto Rico Departamento de Salud,0,Insufficient Data,May-24-2014,2013-14,21
Virgin Islands,http://doh.vi.gov/,Influenza,0,Insufficient Data,May-24-2014,2013-14,21
Connecticut,http://www.ct.gov/dph/cwp/view.asp?a=3136&q=410788,Flu Statistics,0,Insufficient Data,May-31-2014,2013-14,22
Puerto Rico,http://www.salud.gov.pr/Estadisticas-Registros-y-Publicaciones/Pages/Influenza.aspx,Puerto Rico Departamento de Salud,0,Insufficient Data,May-31-2014,2013-14,22
Virgin Islands,http://doh.vi.gov/,Influenza,0,Insufficient Data,May-31-2014,2013-14,22
Idaho,http://www.healthandwelfare.idaho.gov/Health/DiseasesConditions/SeasonalInfluenzaFlu/tabid/327/Default.aspx,Influenza,0,Insufficient Data,Jun-07-2014,2013-14,23
District of Columbia,http://doh.dc.gov/page/influenza-season,Influenza Information,0,Insufficient Data,Jun-07-2014,2013-14,23


[1] "# of Rows in Dataframe: 16668"

[1] "Dataframe Size: 0.9 Mb"

In [36]:
#display the data 
#df.flu.census %>% head(10)
#sprintf("# of Rows in Dataframe: %s", nrow(df.flu.census))
#sprintf("Dataframe Size: %s", format(object.size(df.flu.census), units = "MB"))

In [54]:
#columns = c("REGION TYPE","REGION","YEAR","WEEK","AGE 0-4","AGE 25-49","AGE 25-64","AGE 5-24","AGE 50-64","AGE 65","ILITOTAL","TOTAL PATIENTS")

columns = c("statename","activity_level","activity_level_label","weekend", "season", "weeknumber")

# select() require column indices that we find trough which() 

df <- df.state %>% select(which(names(df.state) %in% columns))

df %>% head(10)
sprintf("# of Rows in Dataframe: %s", nrow(df))
sprintf("Dataframe Size: %s", format(object.size(df), units = "MB"))

statename,activity_level,activity_level_label,weekend,season,weeknumber
Virgin Islands,0,Insufficient Data,Oct-09-2010,2010-11,40
Virgin Islands,0,Insufficient Data,Oct-16-2010,2010-11,41
Virgin Islands,0,Insufficient Data,Oct-23-2010,2010-11,42
Virgin Islands,0,Insufficient Data,Nov-27-2010,2010-11,47
Virgin Islands,0,Insufficient Data,Dec-04-2010,2010-11,48
Virgin Islands,0,Insufficient Data,Dec-11-2010,2010-11,49
Virgin Islands,0,Insufficient Data,Dec-18-2010,2010-11,50
Virgin Islands,0,Insufficient Data,Dec-25-2010,2010-11,51
Virgin Islands,0,Insufficient Data,Jan-01-2011,2010-11,52
Virgin Islands,0,Insufficient Data,Jan-08-2011,2010-11,1


[1] "# of Rows in Dataframe: 16668"

[1] "Dataframe Size: 0.7 Mb"

All-Caps to proper case. (see this [Stack Overflow question](http://stackoverflow.com/questions/15776732/how-to-convert-a-vector-of-strings-to-title-case))

In [55]:
proper_case <- function(x) {
    return (gsub("\\b([A-Z])([A-Z]+)", "\\U\\1\\L\\2" , x, perl = TRUE))
}

# df <- df %>% mutate(STATENAME = proper_case(STATENAME),
# ACTIVITYESTIMATE = proper_case(ACTIVITYESTIMATE),
#                 WEEKEND = proper_case(WEEKEND))

df %>% head(10)

statename,activity_level,activity_level_label,weekend,season,weeknumber
Virgin Islands,0,Insufficient Data,Oct-09-2010,2010-11,40
Virgin Islands,0,Insufficient Data,Oct-16-2010,2010-11,41
Virgin Islands,0,Insufficient Data,Oct-23-2010,2010-11,42
Virgin Islands,0,Insufficient Data,Nov-27-2010,2010-11,47
Virgin Islands,0,Insufficient Data,Dec-04-2010,2010-11,48
Virgin Islands,0,Insufficient Data,Dec-11-2010,2010-11,49
Virgin Islands,0,Insufficient Data,Dec-18-2010,2010-11,50
Virgin Islands,0,Insufficient Data,Dec-25-2010,2010-11,51
Virgin Islands,0,Insufficient Data,Jan-01-2011,2010-11,52
Virgin Islands,0,Insufficient Data,Jan-08-2011,2010-11,1


# Filtering the data

let's file the data frame by "Local Activity" to aggregate some interesting statistics

In [74]:
# grepl() is the best way to do in-text search
#df_flu <- df %>% filter(grepl("Local Activity", ACTIVITYESTIMATE))

df.flu.activity <- df %>% filter(activity_level > 0)

df.flu.activity %>% head(10)
sprintf("# of Rows in Dataframe: %s", nrow(df.flu.activity))
sprintf("Dataframe Size: %s", format(object.size(df.flu.activity), units = "MB"))

statename,activity_level,activity_level_label,weekend,season,weeknumber
New York City,1,Minimal,May-17-2014,2013-14,20
Alabama,1,Minimal,May-24-2014,2013-14,21
Alaska,1,Minimal,May-24-2014,2013-14,21
Arizona,1,Minimal,May-24-2014,2013-14,21
Arkansas,1,Minimal,May-24-2014,2013-14,21
California,1,Minimal,May-24-2014,2013-14,21
Mississippi,1,Minimal,May-17-2014,2013-14,20
Missouri,1,Minimal,May-17-2014,2013-14,20
Montana,1,Minimal,May-17-2014,2013-14,20
Nebraska,1,Minimal,May-17-2014,2013-14,20


[1] "# of Rows in Dataframe: 15878"

[1] "Dataframe Size: 0.6 Mb"

## Flue Over Time

In [83]:
 as.Date("May-17-2014","%b-%d-%Y")

[1] "2014-05-17"

In [84]:
df.flu.weekend <-df.flu.activity %>%
                    mutate(weekend = as.Date(weekend,"%b-%d-%Y")) %>% #convert to proper date for analysis
                    group_by(weekend) %>%
                    summarize(count = n()) %>%
                    arrange(weekend)

df.flu.weekend %>% head(10)

ERROR: Error in mutate_impl(.data, dots): Evaluation error: unknown encoding: -14.


In [73]:
plot <- ggplot(df_flu_daily, aes(x = WEEKEND, y = count)) +
    geom_line(color = "#F2CA27", size = 0.1) +
    geom_smooth(color = "#1A1A1A") +
    fte_theme() +
    scale_x_date(breaks = date_breaks("2 years"), labels = date_format("%Y")) +
    labs(x = "Weekend of the obseration", y = "# of Local Activity", title = "Weekly Flu Activity in the US from 2015 – 2017")

max_save(plot, "US-ILI-when-1", "US OpenData")

ERROR: Error in ggplot(df_flu_daily, aes(x = WEEKEND, y = count)): object 'df_flu_daily' not found
